In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Celiac_Disease"
cohort = "GSE20332"

# Input paths
in_trait_dir = "../../input/GEO/Celiac_Disease"
in_cohort_dir = "../../input/GEO/Celiac_Disease/GSE20332"

# Output paths
out_data_file = "../../output/preprocess/Celiac_Disease/GSE20332.csv"
out_gene_data_file = "../../output/preprocess/Celiac_Disease/gene_data/GSE20332.csv"
out_clinical_data_file = "../../output/preprocess/Celiac_Disease/clinical_data/GSE20332.csv"
json_path = "../../output/preprocess/Celiac_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Primary human leukocyte RNA expression of unrelated Dutch and UK individuals"
!Series_summary	"Multiple common variants for celiac disease influencing immune gene expression"
!Series_summary	"The goal of this study was to study the effect of genetic variation on gene expression of untouched primary leucocytes."
!Series_summary	"We obtained peripheral blood RNA from unrelated Dutch and UK individuals using PAXgene tubes."
!Series_summary	"We performed a second-generation genome wide association study of 4,533 celiac disease cases and 10,750 controls. We genotyped 113 selected SNPs with PGWAS<10-4, and 18 SNPs from 14 known loci, in a further 4,918 cases and 5,684 controls. Variants from 13 new regions reached genome wide significance (Pcombined<5x10-8), most contain immune function genes (BACH2, CCR4, CD80, CIITA/SOCS1/CLEC16A, ICOSLG, ZMIZ1) with ETS1, RUNX3, THEMIS and TNFRSF14 playing key roles in thymic T cell selection. A further 13 regions ha

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any, List

# 1. Gene Expression Data Availability
# From the background information, we can see this is a gene expression dataset
# that studied "the effect of genetic variation on gene expression of untouched primary leucocytes"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary
print("Sample characteristics dictionary:", {0: ['tissue: Peripheral blood']})

# 2.1 Data Availability
# The sample characteristics dictionary doesn't contain trait (celiac disease), age, or gender information
trait_row = None  # No information about celiac disease status
age_row = None    # No age information
gender_row = None # No gender information

# 2.2 Data Type Conversion
# Even though we don't have data, we need to define these conversion functions

def convert_trait(value):
    """Convert trait value to binary (1 for celiac disease, 0 for control)"""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'celiac' in value or 'case' in value or 'patient' in value or 'cd' in value:
        return 1
    elif 'control' in value or 'healthy' in value or 'normal' in value:
        return 0
    return None

def convert_age(value):
    """Convert age value to continuous (float)"""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    else:
        value = value.strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if value in ['female', 'f', 'woman']:
        return 0
    elif value in ['male', 'm', 'man']:
        return 1
    return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None

# Save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None (clinical data not available)


Sample characteristics dictionary: {0: ['tissue: Peripheral blood']}


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Celiac_Disease/GSE20332/GSE20332_series_matrix.txt.gz


Gene data shape: (22185, 229)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343292', 'ILMN_1343293', 'ILMN_1343294',
       'ILMN_1651209', 'ILMN_1651217', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651234', 'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237',
       'ILMN_1651238', 'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260',
       'ILMN_1651261', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Illumina probe IDs (ILMN_), not human gene symbols
# They need to be mapped to standard gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1698220', 'ILMN_1810835', 'ILMN_1782944', 'ILMN_1692858', 'ILMN_1668162'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'RefSeq', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_13666', 'ILMN_10478', 'ILMN_27850', 'ILMN_10309', 'ILMN_7652'], 'Transcript': ['ILMN_13666', 'ILMN_175835', 'ILMN_27850', 'ILMN_10309', 'ILMN_7652'], 'ILMN_Gene': ['PHTF2', 'SPRR3', 'GPR37L1', 'FBXO25', 'DGAT2L3'], 'Source_Reference_ID': ['NM_020432.2', 'NM_005416.1', 'NM_004767.2', 'NM_012173.3', 'NM_001013579.1'], 'RefSeq_ID': ['NM_020432.2', 'NM_005416.1', 'NM_004767.2', 'NM_012173.3', 'NM_001013579.1'], 'Entrez_Gene_ID': [57157.0, 6707.0, 9283.0, 26260.0, 158833.0], 'GI': [40254932.0, 4885606.0, 31377792.0, 34878756.0, 61888901.0], 'Accession': ['NM_020432.2', 'NM_005416.1', 'NM_004767.2', 'NM_012173.3', 'NM_001013579.1'], 'Symbol': ['PHTF2', 'SPRR3', 'GPR37L1', 'FBXO25', 'DGAT2L3'], 'Protein_Prod

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the annotation data correspond to probe IDs and gene symbols
# From the preview, we can see that the 'ID' column has the ILMN_ identifiers matching the gene expression data
# The 'Symbol' column contains the gene symbols we need to map to

# 2. Get a gene mapping dataframe using the get_gene_mapping function from the library
prob_col = 'ID'      # Column with probe identifiers (ILMN_*)
gene_col = 'Symbol'  # Column with gene symbols (e.g., PHTF2)

mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping data shape: {mapping_data.shape}")
print("Sample of mapping data:")
print(mapping_data.head())

# 3. Convert probe-level measurements to gene-level expression data using the mapping
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])


Gene mapping data shape: (22185, 2)
Sample of mapping data:
             ID     Gene
0  ILMN_1698220    PHTF2
1  ILMN_1810835    SPRR3
2  ILMN_1782944  GPR37L1
3  ILMN_1692858   FBXO25
4  ILMN_1668162  DGAT2L3
Gene expression data shape after mapping: (17609, 229)
First few gene symbols after mapping:
Index(['A1BG', 'A2BP1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS',
       'AACS', 'AADAC'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {gene_data_normalized.shape}")
print("Sample of normalized gene data:")
print(gene_data_normalized.iloc[:5, :5])

# Save gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Since we determined in Step 2 that trait_row is None (no clinical data available),
# we cannot perform data linking or further processing

# Create an empty DataFrame to represent the lack of linked data
linked_data = pd.DataFrame()

# 5. Conduct quality check and save the cohort information
# We already know is_trait_available is False since trait_row is None
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,  # No trait data available
    is_biased=False,  # Set to False instead of None for is_trait_available=False case
    df=linked_data,
    note="Dataset contains gene expression data but lacks phenotype information about Celiac Disease status"
)

# 6. Since the data is not usable (no trait data), we do not save linked data
print("Data is not usable for trait analysis because trait information is missing.")

Normalized gene data shape: (16991, 229)
Sample of normalized gene data:
        GSM509318  GSM509319  GSM509320  GSM509321  GSM509322
Gene                                                         
A1BG    -0.732172  -0.628766  -0.762766  -0.616788  -0.665529
A2M     -0.630958  -0.610310  -0.500944  -0.512666  -0.658750
A2ML1   -0.573289  -0.490310  -0.516602  -0.550659  -0.517589
A4GALT  -0.543558  -0.544570  -0.338999  -0.565004  -0.530494
A4GNT   -0.504837  -0.519564  -0.538515  -0.609235  -0.551677


Gene expression data saved to ../../output/preprocess/Celiac_Disease/gene_data/GSE20332.csv
Abnormality detected in the cohort: GSE20332. Preprocessing failed.
Data is not usable for trait analysis because trait information is missing.
